In [1]:
import requests
import math
import numpy as np
from astropy.time import Time
from astropy.table import QTable
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
#plt.rc('font', family='DejaVu Sans', serif='Times')
#plt.rc('text', usetex=True)
from pprint import pprint
from datetime import datetime
import re
from astroquery.jplhorizons import Horizons

This API provides access to the JPL/SSD small-body mission design suite. The following operation modes are available:
* Mode T (mission extension) - retrieves the list of small bodies that come closest (or within a prescribed distance) to a user-defined orbit during a certain period of time. This is a crude filter for finding potential candidates for mission extensions.

The reference heliocentric orbit is defined by providing the corresponding set of cometary elements.

Mode T - Request list of small bodies that come closest (or within a prescribed distance) to a user-specified heliocentric orbit (assumes two-body dynamics). Proxy for easiest-to-reach targets for an extended mission phase.

Query Examples:
Request the 100 objects that come within 0.001 au of a user-specified orbit between 2020-1-1 and 2020-10-10.
* https://ssd-api.jpl.nasa.gov/mdesign.api?ec=0.2056408220896557&qr=0.3074958016246215&tp=2459067.6508400026&om=48.30597718083336&w=29.18348714438387&in=7.003733902930839&jd0=2458849.5&jdf=2459132.5&maxout=100&maxdist=0.0010 

Output:
In mode T, the JSON payload contains the list of objects that come closest to the user-specified orbit.

    md_constraints - summarizes the user-specified constraints (orbit parameters, time span, maximum distance, and maximum number of records)
    ec - eccentricity of the reference orbit.
    qr - perihelion distance of the reference orbit.
    tp - time of perihelion passage of the reference orbit (JD).
    in - inclination of the reference orbit (deg).
    om - longitude of the ascending node of the reference orbit (deg).
    w - argument of periapsis of the reference orbit (deg).
    maxdist - (if maxdist was provided) maximum admissible close-approach distance (au).
    maxout - maximum number of output records.
    jd0 - start of the time span (JD).
    jdf - end of the time span (JD). Time span must not be longer than one year.
    count - number of objects in the output list.
    fields - list of fields in the output table:
    full_name - object full name.
    date - date of closest approach (calendar).
    jd - date of closest approach (JD).
    min_dist_au - close-approach distance (au).
    min_dist_km - close-approach distance (km).
    rel_vel - relative velocity at closest approach (km/s).
    class - small-body orbit class.
    H - absolute magnitude.
    condition_code - orbit condition code.
    neo - near-Earth object flag.
    pha - potentially hazardous asteroid flag.
    sats - number of satellites.
    spkid - SPK object id.
    pdes - primary designation.
    data - output data arranged in an array where each entry corresponds to a small body and the fields are defined by fields.

[Reference of the API construction, data input and output](https://ssd-api.jpl.nasa.gov/doc/mdesign.html)

In [2]:
def get_horizons_ephemerides_elements(name,pov,epoch_start):
    
    # step: step size, [10m, 1d, 1y]
    
    if pov.lower() == 'sun':
        loc = '500@10' # position relative to the sun
    elif pov.lower() == 'goldstone':
        loc = '257' # from goldstone
    elif pov.lower() == 'maunakea':
        loc = '568' # maunakea
    else:
        print('Not Valid Location Point Of View')
    
    # Process to get homogeneity from main script full name '2012QD8' to a valid name for Horizon call '2012 QD8'
    if len(re.findall('([0-9])', name)) <= 4: # 4 is the min numbers in every name, the date year of discovery
        r = re.compile("([0-9]+)([a-zA-Z]+)").match(name)
        k1 = r.group(1) # the date of the name
        k2 = r.group(2) # the code of the date
        valid_name = k1 + " " + k2 
    else:
        r = re.compile("([0-9]+)([a-zA-Z]+)([0-9]+)").match(name)
        k1 = r.group(1) # the date of the name
        k2 = r.group(2) # the code of the date
        k3 = r.group(3) # id after the letters
        valid_name = k1 + " " + k2 + k3
    
    # always a day after the input, anyway you consider the moment of input, the first of the data output extracted
    epoch_start
    chunks = epoch_start.split('-')
    chunks2 = int(chunks[2]) + 1 # add 1 day
    list_string = [chunks[0], chunks[1], str(chunks2)]
    epoch_stop = '-'.join(list_string)

    step_size = '1d'
    
    obj = Horizons(id=valid_name, 
               location=loc, 
               epochs={'start': epoch_start, 'stop':epoch_stop,
                       'step': step_size})
    
    # refsystem = 'J2000', # Element reference system for geometric and astrometric quantities
    # refplane = 'ecliptic' #ecliptic and mean equinox of reference epoch
    data = obj.elements(refsystem = 'J2000',refplane = 'ecliptic')

    len_cols = 7 # jd,ec,qr,tp,incl,OM,om
    adata =  np.zeros([1,len_cols]) 
    # always assign the first row of output data -> the first date required!
    #for row in range(len_rows):
    adata[0,0] = data[0][5] # 6th column of data -> e, eccentricity (-)
    adata[0,1] = data[0][6] # 7th column of data -> qr, periapsis distance (AU)
    adata[0,2] = data[0][10] # 11th column of data -> tp, time of periapsis (JD)
    adata[0,3] = data[0][7] # 8th column of data -> incl, inclination (deg)
    adata[0,4] = data[0][8] # 10th column of data -> OM, longitude of Asc. Node (deg)
    adata[0,5] = data[0][9] # 11th column of data -> om, argument of periapsis (deg)
    adata[0,6] = data[0][1] # 2nd column of the data extracted -> jd of evaluation
        
    return adata

In [4]:
# Mode T - Request list of small bodies that come closest (or within a prescribed distance) to a user-specified heliocentric 
# orbit (assumes two-body dynamics). Proxy for easiest-to-reach targets for an extended mission phase.
# example url
# https://ssd-api.jpl.nasa.gov/mdesign.api?ec=0.2056408220896557&qr=0.3074958016246215&tp=2459067.6508400026&
# om=48.30597718083336&w=29.18348714438387&in=7.003733902930839&jd0=2458849.5&jdf=2459132.5&maxout=100&maxdist=0.0010

def get_close_approach_to_asteroid(orb_params,jd0,jdf,n_object_requested,distance_within):
    # The reference heliocentric orbit is defined by providing the corresponding set of cometary elements.
    # orb_params:            array containing the orbital parameters required to run the query
    #     ec:                eccentricity [>0]
    #     qr:                perihelion distance [>0]
    #     tp:                time of perihelion passage (JD)
    #     incl:              inclination (deg) [0,180]
    #     OM:                longitude of the ascending node (deg) [0,360]
    #     om:                argument of periapsis (deg) [0,360]
    # jd0:                   beginning of the requested time span (JD)
    # jdf:                   end of the requested time span (JD). Time span must not be longer than one year
    # n_object_requested:    maximum number of records to be returned
    # distance_within:       ignore objects with distance of closest approach greater than "distance_within" [>0, optional]
    
    # Extraction of inputs
    ec = orb_params[0,0]
    qr = orb_params[0,1]
    tp = orb_params[0,2]
    incl = orb_params[0,3]
    OM = orb_params[0,4]
    om = orb_params[0,5]
    
    # Construction of the HTTP request
    url_base = 'https://ssd-api.jpl.nasa.gov/mdesign.api'
    url = f'{url_base}?ec={str(ec)}&qr={str(qr)}&tp={str(tp)}&in={str(incl)}&om={str(OM)}&w={str(om)}&jd0={str(jd0)}&jdf={str(jdf)}&maxout={str(n_object_requested)}&maxdist={str(distance_within)}'
    r = requests.get(url)
    data = r.json()
    
    return data

In [6]:
name = '2009TD17'
PointOfView = 'Sun'
epoch_start = '2032-12-01'## end of mission on SC 2, with CoastingTime on 2nd ast = CT1 1st ast

data_2009TD17 = get_horizons_ephemerides_elements(name,PointOfView,epoch_start)
data_2009TD17

array([[2.20002242e-01, 8.78987570e-01, 2.46335280e+06, 7.93565618e-02,
        2.17328689e+02, 8.35794392e+01, 2.46356750e+06]])

In [7]:
jd0 = data_2009TD17[0,6] # date of the required orbit as before
jdf = jd0 + 100 # must be max 1 year more than jd0
n_object_requested = 100
distance_within = 0.05 # I suppose in AU, 0.0026 it's approx 1 LD
CAD_2009TD17 = get_close_approach_to_asteroid(data_2009TD17,jd0,jdf,n_object_requested,distance_within)

In [8]:
dist_km = CAD_2009TD17['data'][1][4] 
rel_vel_km_s = CAD_2009TD17['data'][1][5]

In [9]:
def get_CAD_params(cad_input, idx_element):
    # obtain close approach data interesting to be analysed
    adata =  np.zeros([1,6]) 
    if cad_input['data'][1][9] == 'Y':
        # vector with the object names
        aname1 = cad_input['data'][idx_element][0] # name
        aname2 = aname1.strip()
        aname3 = aname2.split('(')
        aname4 = aname3[1].split(')')
        aname = aname4[0]
        
        adata[0,0] = cad_input['data'][idx_element][2] # jd of encounter
        adata[0,1] = cad_input['data'][idx_element][4] # distance close approach km
        adata[0,2] = cad_input['data'][idx_element][5] # relative velocity km/s
        adata[0,3] = cad_input['data'][idx_element][7] # absolute magnitude H
        adata[0,4] = cad_input['data'][idx_element][8] # orbit condition code
        if cad_input['data'][idx_element][10] == 'Y':
            adata[0,5] = 1 # pha flag true -> 1
        elif cad_input['data'][idx_element][10] == 'N':
            adata[0,5] = 0 # pha flag false -> 0
            
    return aname, adata
    

In [11]:
cad_objects = []
cad_params =  np.zeros([int(CAD_2009TD17['count']),6]) 
idx = 0;
for i in range(0,int(CAD_2009TD17['count'])):
    obj, params = get_CAD_params(CAD_2009TD17,i)
    if obj != '2009 TD17':
        cad_objects.append(obj)
        cad_params[idx,0:6] = params
        idx = idx + 1

In [12]:
cad_params

array([[6.36670000e+04, 2.85336404e+06, 7.52504730e-04, 2.79000000e+01,
        7.00000000e+00, 0.00000000e+00],
       [6.36670000e+04, 3.25820711e+06, 4.11954267e-03, 2.31000000e+01,
        1.00000000e+00, 0.00000000e+00],
       [6.35670000e+04, 3.68937972e+06, 1.14474144e-03, 2.62000000e+01,
        7.00000000e+00, 0.00000000e+00],
       [6.36570000e+04, 3.83192963e+06, 5.55130105e-03, 2.22000000e+01,
        7.00000000e+00, 0.00000000e+00],
       [6.36480000e+04, 4.18210317e+06, 1.33066212e-02, 2.38000000e+01,
        9.00000000e+00, 0.00000000e+00],
       [6.36200000e+04, 4.83503831e+06, 9.47634094e-03, 2.48000000e+01,
        9.00000000e+00, 0.00000000e+00],
       [6.36450000e+04, 5.14778802e+06, 4.83272586e-03, 2.49290000e+01,
        8.00000000e+00, 0.00000000e+00],
       [6.35810000e+04, 5.58164810e+06, 6.99683844e-03, 2.19000000e+01,
        5.00000000e+00, 1.00000000e+00],
       [6.36350000e+04, 6.11457165e+06, 9.32758042e-03, 1.62900000e+01,
        1.00000000e+00, 

In [ ]:
name = '2021JE1'
PointOfView = 'Sun'
epoch_start = '2032-12-15'## end of mission on SC 2, with CoastingTime on b_th ast = CTa a_th ast

data_2021JE1 = get_horizons_ephemerides_elements(name,PointOfView,epoch_start)
data_2021JE1

In [ ]:
jd0 = data_2021JE1[0,6] # date of the required orbit as before
jdf = jd0 + 100 # must be max 1 year more than jd0
n_object_requested = 100
distance_within = 0.05 # I suppose in AU, 0.0026 it's approx 1 LD
CAD_2021JE1 = get_close_approach_to_asteroid(data_2021JE1,jd0,jdf,n_object_requested,distance_within)

In [ ]:
cad_objects2 = []
cad_params2 =  np.zeros([int(CAD_2021JE1['count']),6]) 
for i in range(0,int(CAD_2021JE1['count'])):
    obj, params = get_CAD_params(CAD_2021JE1,i)
    cad_objects2.append(obj)
    cad_params2[i,0:6] = params

In [ ]:
cad_objects2

In [ ]:
cad_params2